In [ ]:
!pip install qiskit

In [ ]:
import csv
import math
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance
from qiskit import Aer
from qiskit_machine_learning.kernels import QuantumKernel
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# -------------------
# 1. Load CSV manually
# -------------------
def load_csv(file_path):
    with open(file_path, "r") as f:
        reader = csv.DictReader(f)
        data = [row for row in reader]
    return data

# -------------------
# 2. Encode categorical values manually
# -------------------
def encode_categorical(data, column, mapping):
    for row in data:
        row[column] = mapping[row[column]]

# -------------------
# 3. Compute derived metrics
# -------------------
def compute_metrics(data):
    for row in data:
        training_intensity = float(row["Training_Intensity"])
        training_hours = float(row["Training_Hours_Per_Week"])
        recovery_days = float(row["Recovery_Days_Per_Week"])
        fatigue_score = float(row["Fatigue_Score"])
        matches = float(row["Match_Count_Per_Week"])
        
        # Load_Balance_Score
        load_balance = (
            (recovery_days * 10)
            + (10 - abs(training_intensity - 5) * 2)
            + (20 - abs(training_hours - 12) * 1.5)
        )
        row["Load_Balance_Score"] = load_balance
        
        # ACL_Risk_Score
        acl_risk = (
            (fatigue_score * 10)
            + (matches * 15)
            - (load_balance * 0.5)
        )
        row["ACL_Risk_Score"] = acl_risk

# -------------------
# 4. Scale features manually (min-max scaling)
# -------------------
def scale_features(data, feature_names):
    for feature in feature_names:
        values = [float(row[feature]) for row in data]
        min_val, max_val = min(values), max(values)
        for row in data:
            row[feature] = (float(row[feature]) - min_val) / (max_val - min_val)

# -------------------
# 5. Prepare data for quantum model
# -------------------
def prepare_data(data, feature_names, target_name):
    X = []
    y = []
    for row in data:
        X.append([float(row[f]) for f in feature_names])
        y.append(int(row[target_name]))
    return X, y

# -------------------
# 6. Quantum kernel SVM
# -------------------
def train_quantum_svm(X_train, X_test, y_train, y_test):
    feature_map = ZZFeatureMap(feature_dimension=len(X_train[0]), reps=2, entanglement='full')
    backend = Aer.get_backend('statevector_simulator')
    quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=QuantumInstance(backend))
    
    svc = SVC(kernel=quantum_kernel.evaluate)
    svc.fit(X_train, y_train)
    preds = svc.predict(X_test)
    
    print("Accuracy:", accuracy_score(y_test, preds))
    print(classification_report(y_test, preds))

# -------------------
# Main execution
# -------------------
data = load_csv("/mnt/data/collegiate_athlete_injury_dataset.csv")

# Encode categorical
encode_categorical(data, "Gender", {"Male": 0, "Female": 1})
encode_categorical(data, "Position", {"Guard": 0, "Forward": 1, "Center": 2})

# Compute derived metrics
compute_metrics(data)

# Scale features
feature_names = [
    "Age", "Gender", "Height_cm", "Weight_kg", "Position",
    "Training_Intensity", "Training_Hours_Per_Week", "Recovery_Days_Per_Week",
    "Match_Count_Per_Week", "Rest_Between_Events_Days", "Load_Balance_Score",
    "ACL_Risk_Score", "Fatigue_Score", "Performance_Score", "Team_Contribution_Score"
]
scale_features(data, feature_names)

# Prepare data
X, y = prepare_data(data, feature_names, "Injury_Indicator")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train & evaluate quantum SVM
train_quantum_svm(X_train, X_test, y_train, y_test)


ImportError: cannot import name 'QuantumInstance' from 'qiskit.utils' (c:\Users\rpall\anaconda3\envs\tes\Lib\site-packages\qiskit\utils\__init__.py)